In [ ]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings('ignore')
path="../../../ukb/data/"

import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

from OSGridConverter import grid2latlong

'''
Helper function to find out columns containing specific text
'''
def findcols(df,text):
    return [col for col in df.columns if text in col]

In [ ]:
dis_vars=['n_eid','Age_Today','AD','PD','Diag_Date_AD','years_bef_diagAD','Age_Diag_AD',
                           'date_of_attending_assessment_centre_f53', 'age_when_attended_assessment_centre_f21003',
                           'EPIL','DIAB','CELIAC','CERVASC','CERVASC_bef','CERVASCALL_bef','EPIL_bef','DIAB_bef']

shap_vars2=['eid','sex_f31','Genotype','particulate_matter_air_pollution_pm25_2010_f24006','airpoll_24006','airpoll_24007',
       'mean_time_to_correctly_identify_matches_f20023','rounded_matches_f20023','urea_f30670',
       'number_of_treatmentsmedications_taken_f137','nitrogen_dioxide_air_pollution_2006_f24017',
       'sleeplessness_insomnia_f1200',
       'particulate_matter_air_pollution_pm25_absorbance_2010_f24007','mean_corpuscular_volume_f30040',
       'never_eat_eggs_dairy_wheat_sugar_f6144',
       'uk_east_coord',
       'parental_ad_status','qualifications_f6138',
       'uk_north_coord',
       'particulate_matter_air_pollution_pm10_2010_f24005',
       'time_spent_driving_f1090', 'creactive_protein_f30710',
       'alanine_aminotransferase_f30620','aspartate_aminotransferase_f30650',
       'answered_sexual_history_questions_f2129',
       'neutrophill_count_f30140', 'carer_support_indicators_f41214',
       'drive_faster_than_motorway_speed_limit_f1100',
       'history_of_psychiatric_care_on_admission_f41218',
       'particulate_matter_air_pollution_2510um_2010_f24008',
       'frequency_of_solariumsunlamp_use_f2277',
       'hand_grip_strength_left_f46', 'glycated_haemoglobin_hba1c_f30750',
       'frequency_of_tenseness_restlessness_in_last_2_weeks_f2070',
       'forced_expiratory_volume_in_1second_fev1_best_measure_f20150',
            'diabetes_diagnosed_by_doctor_f2443','overall_health_rating_f2178','processed_meat_intake_f1349',
           'number_of_daysweek_walked_10_minutes_f864','duration_of_walks_f874','sleep_duration_f1160','testosterone_f30850',
           'qualif_score']



shapvars3=['difference_in_mobile_phone_use_compared_to_two_years_previously_f1140',
       'drive_faster_than_motorway_speed_limit_f1100',
       'drive_faster_than_motorway_speed_limit_f1100',
       'frequency_of_tenseness_restlessness_in_last_2_weeks_f2070',
       'met_minutes_per_week_for_walking_f22037',
       'never_eat_eggs_dairy_wheat_sugar_f6144',
       'nitrogen_dioxide_air_pollution_2006_f24017',
       'nitrogen_oxides_air_pollution_2010_f24004',
       'particulate_matter_air_pollution_2510um_2010_f24008',
       'particulate_matter_air_pollution_pm10_2007_f24019',
       'particulate_matter_air_pollution_pm10_2010_f24005',
       'particulate_matter_air_pollution_pm25_2010_f24006',
       'particulate_matter_air_pollution_pm25_absorbance_2010_f24007',
       'sleeplessness_insomnia_f1200', 'tea_intake_f1488',
       'time_spent_driving_f1090',
       'traffic_intensity_on_the_nearest_major_road_f24011',
       'usual_walking_pace_f924', 'water_intake_f1528']


shap_vars2=np.unique(np.concatenate([shap_vars2,shapvars3]))


shap_vars=['year_of_birth_f34','place_of_birth_in_uk_east_coordinate_f130',\
           'number_of_treatmentsmedications_taken_f137',
 'time_spent_using_computer_f1080',
 'sleeplessness_insomnia_f1200',
 'ever_had_samesex_intercourse_f2159',
 'longstanding_illness_disability_or_infirmity_f2188',
 'frequency_of_solariumsunlamp_use_f2277',
 'diabetes_diagnosed_by_doctor_f2443',
 'mothers_age_at_death_f3526',
 'never_eat_eggs_dairy_wheat_sugar_f6144',
 'mean_time_to_correctly_identify_matches_f20023',
 'particulate_matter_air_pollution_pm25_2010_f24006',
 'carer_support_indicators_f41214',
 'death',
 'parental_ad_status',
 'Genotype']

vars_analyse=['eid','Genotype','recent_feelings_of_depression_f20510','sleeplessness_insomnia_f1200','parental_ad_status',
      'particulate_matter_air_pollution_pm25_2010_f24006',
      'number_of_symbol_digit_matches_made_correctly_f20159','sex_f31',
      'type_of_special_diet_followed_f20086','ever_used_hormonereplacement_therapy_hrt_f2814'
      ]


qualmap={'None of the above':0,'Prefer not to answer':0,'O levels/GCSEs or equivalent':1,\
'NVQ or HND or HNC or equivalent':2,'NVQ or HND or HNC or equivalent' :3,\
'NVQ or HND or HNC or equivalent':4,'CSEs or equivalent':5,\
'Other professional qualifications eg: nursing, teaching':6,'A levels/AS levels or equivalent':7,\
'College or University degree':8 }

In [ ]:
def combineanalyse(df1,df2):
    df_out=pd.merge(df1,df2,left_on='eid',right_on='n_eid',how='left')
    return df_out    


from scipy import stats
def pvalue_slopes(df,var):
    
    mask_aspnnull=(pd.notnull(ukb_shap[var]))
    apoe_mask=(df['Genotype']=="e4/e4")|(df['Genotype']=="e3/e4")|(df['Genotype']=="e2/e4")
    
    rng_apoe=(df[mask_aspnnull&apoe_mask][var].max()-df[mask_aspnnull&apoe_mask][var].min())
    
    slope_ap, intercept_ap, r_value_ap, p_value_ap, std_err_ap = \
    stats.linregress(ukb_shap[mask_aspnnull&apoe_mask][var],ukb_shap[mask_aspnnull&apoe_mask]['AD'])
    
    slope_ap_fin=slope_ap*rng_apoe
    
    x=ukb_shap[mask_aspnnull&apoe_mask][var]
    tinv = lambda p, df: abs(t.ppf(p/2, df))
    ts = tinv(0.05, len(x)-2)
    apoe_CI=ts*std_err_ap*rng_apoe

    
    #from scipy.stats import t
    #tinv = lambda p, df: abs(t.ppf(p/2, df)
    
        #print(slope_ap)
    rng_nonapoe=ukb_shap[mask_aspnnull&~apoe_mask][var].max()-ukb_shap[mask_aspnnull&~apoe_mask][var].min()
    
    slope_nonap, intercept_nonap, r_value_nonap, p_value_nonap, std_err_nonap = \
    stats.linregress(ukb_shap[mask_aspnnull&~apoe_mask][var],ukb_shap[mask_aspnnull&~apoe_mask]['AD'])
    #print(slope_nonap)
    
    slope_nonap_fin=slope_nonap*rng_nonapoe
    
    x=ukb_shap[mask_aspnnull&~apoe_mask][var]
    tinv = lambda p, df: abs(t.ppf(p/2, df))
    ts = tinv(0.05, len(x)-2)
    nonapoe_CI=ts*std_err_ap*rng_nonapoe

    numerator = slope_ap - slope_nonap
    denominator = pow((pow(std_err_ap,2) + pow(std_err_nonap,2)), 1/2)
    z=numerator/denominator  
    #print(z)

    p_value = stats.norm.sf(abs(z))
    print(slope_ap_fin-slope_nonap_fin)
    
    
    return slope_ap_fin,slope_nonap_fin,p_value,apoe_CI,nonapoe_CI,p_value_ap,p_value_nonap


def lines_conf(var,df=ukb_shap,minsamp=2000,regplot=0,rounded=1,factdiv=1,riskscore=0,rat='subt',e4e4=0):
    ukb_shap_var=df[['eid',var,'APOE','Genotype','AD','risk_score_norm']]
    
    mask=(ukb_shap_var['APOE']==1)
    maske4=(ukb_shap_var['Genotype']=='e4/e4')
    ukb_shap_var['APOE Status']=ukb_shap_var['APOE']
    
    ukb_shap_var['APOE Status'][~mask]='Non APOE4 Carriers'
    if e4e4==1:
        ukb_shap_var['APOE Status'][mask]='Single APOE4 Carriers'
        ukb_shap_var['APOE Status'][maske4]='APOE4 Carriers e4/e4'
        
    else:
        ukb_shap_var['APOE Status'][mask]='APOE4 Carriers'
    
    
    
    ukb_shap_var[var+'rounded']=round(ukb_shap_var[var]/factdiv,rounded)*factdiv
    dfgp=pd.DataFrame(ukb_shap_var.groupby(['APOE Status',var+'rounded']).agg({'AD':['mean','count','sum']})).reset_index()
    dfgp.columns=['APOE Status',var+'rounded','AD Incidence Rate','recs','total_AD']
    mask=(dfgp['recs']>=minsamp)
    dfgp=dfgp[mask]
    ukb_shap_var=pd.merge(ukb_shap_var,dfgp,on=['APOE Status',var+'rounded'],how='inner')
    
    if e4e4==1:
        maskapoe=(dfgp['APOE Status']=='Single APOE4 Carriers')|(dfgp['APOE Status']=='APOE4 Carriers e4/e4')
    else:
        maskapoe=(dfgp['APOE Status']=='APOE4 Carriers')
    
    dfgp_apoe=dfgp[maskapoe]
    dfgp_apoe.rename(columns={'AD Incidence Rate':'APOERate'},inplace=True)
    dfgp_non_apoe=dfgp[~maskapoe]
    dfgp_non_apoe.rename(columns={'AD Incidence Rate':'nonAPOERate'},inplace=True)
    df_gp_apoenon=pd.merge(dfgp_apoe[[var+'rounded','APOERate']],
                           dfgp_non_apoe[[var+'rounded','nonAPOERate']],on=var+'rounded',how='inner')
    
    if rat=='mult':
        yname='Ratio: APOE4 to non APOE4 Incidence'
        df_gp_apoenon[yname]=df_gp_apoenon['APOERate']/df_gp_apoenon['nonAPOERate']
    elif rat=='subt':
        yname='Difference: APOE4 - non APOE4 Incidence'
        df_gp_apoenon[yname]=df_gp_apoenon['APOERate']-df_gp_apoenon['nonAPOERate']
    
    
    plt.show()
    
    if riskscore==0:
        ax=sns.lineplot(data=ukb_shap_var, x=var+'rounded', y='AD',hue='APOE Status',estimator='mean')
        ax.set(xlabel=var,ylabel='AD Incidence Rate')
        plt.show()
    if riskscore==1:
        ax=sns.lineplot(data=ukb_shap_var, x=var+'rounded', y='risk_score_norm',hue='APOE Status',estimator='mean')
        ax.set(xlabel=var,ylabel='AD Risk Score')
        plt.show()
        
    
    if regplot==1:
        #sns.lineplot(data=df_gp_apoenon, x=var+'rounded', y='ratio',estimator='mean')
        sns.regplot(data=df_gp_apoenon, x=var+'rounded', y=yname)
        plt.show()
        
        slope, intercept, r_value, p_value, std_err = \
        stats.linregress(df_gp_apoenon[var+'rounded'],df_gp_apoenon[yname])

        x=df_gp_apoenon[var+'rounded']
        tinv = lambda p, df: abs(t.ppf(p/2, df))
        ts = tinv(0.05, len(x)-2)
        slope_CI=ts*std_err
        print(f"slope (95%): {slope:.6f} +/- {slope_CI:.6f}")
        #sns.regplot(data=dfgp, x=var+'rounded', y='AD Incidence Rate')
        #plt.show()
    else:
        sns.lineplot(data=df_gp_apoenon, x=var+'rounded', y=yname,estimator='mean')

In [ ]:
ukb_tp0=pd.read_pickle('%s%s' % (path,'ukb_tp0.p'))
diseases_bin=pd.read_pickle('%s%s' % (path,'ukb_disease_labels.p'))
outputs_score=pd.read_pickle('%s%s' % (path,'outputs_score.p'))


In [ ]:
ukb_tp0['rounded_matches_f20023']=(ukb_tp0['mean_time_to_correctly_identify_matches_f20023']/50).round()*10
ukb_tp0['airpoll_24006']=ukb_tp0['particulate_matter_air_pollution_pm25_2010_f24006'].round()
ukb_tp0['airpoll_24007']=ukb_tp0['particulate_matter_air_pollution_pm25_absorbance_2010_f24007'].round()
ukb_tp0['uk_north_coord']=(ukb_tp0['place_of_birth_in_uk_north_coordinate_f129']/100000).round()*100000
ukb_tp0['uk_east_coord']=(ukb_tp0['place_of_birth_in_uk_east_coordinate_f130']/100000).round()*100000

ukb_tp0['qualif_score']=ukb_tp0['qualifications_f6138'].map(qualmap)

In [ ]:
maskAD=(diseases_bin['AD']==1)
mask_death=(diseases_bin['death']==1)
mask_ADdiag=(diseases_bin['years_bef_diagAD']>2)
eids_ADDiag=diseases_bin['n_eid'][~mask_ADdiag|(mask_death&~maskAD)]

mask_ukb0_AD=(ukb_tp0['eid'].isin(eids_ADDiag))
print(ukb_tp0[mask_ukb0_AD].shape)

mask_age=(ukb_tp0['Age_Today']>=70)
ukb_tp0_red=ukb_tp0[mask_age&mask_ukb0_AD]
print(ukb_tp0_red.shape)

In [ ]:
ukb_shap=ukb_tp0[[col for col in ukb_tp0.columns if col in shap_vars2 or col in modriskfactors]]
ukb_shap=pd.merge(ukb_shap,diseases_bin[dis_vars],left_on='eid',right_on='n_eid',how='left')
ukb_shap=pd.merge(ukb_shap,score_rebal,on='eid',how='inner')


In [ ]:
apoe_mask=(ukb_shap['Genotype']=="e4/e4")|(ukb_shap['Genotype']=="e3/e4")|(ukb_shap['Genotype']=="e2/e4")

ukb_shap['APOE']=0
ukb_shap['APOE'][(apoe_mask)]=1

In [ ]:
ukb_shap['alan_aspar_rat']=round((ukb_shap['aspartate_aminotransferase_f30650']\
                                  /ukb_shap['alanine_aminotransferase_f30620']),1)
total_AD=ukb_shap['AD'].sum()
total_risk=ukb_shap['risk_score'].sum()
riskfact=total_AD/total_risk
ukb_shap['risk_score_norm']=ukb_shap['risk_score']*riskfact

total_risk_mod=ukb_shap['mean_score_mod'].sum()
riskfact_mod=total_AD/total_risk_mod
ukb_shap['risk_score_norm_mod']=ukb_shap['mean_score_mod']*riskfact_mod

mask=pd.notnull(ukb_shap['Age_Diag_AD'])&(ukb_shap['Age_Diag_AD']!=0)
ukb_shap['Age_Today_Rd']=round(ukb_shap['Age_Today'],0)
ukb_shap['Age_Diag_Rd']=round(ukb_shap['Age_Diag_AD'],0)

uqvalue=ukb_shap['risk_score'].quantile(0.75)
lqvalue=ukb_shap['risk_score'].quantile(0.25)

mask_lq=(ukb_shap['risk_score']<lqvalue)
mask_hq=(ukb_shap['risk_score']>uqvalue)
mak_iqr=(ukb_shap['risk_score']>=lqvalue)&(ukb_shap['mean_score_mod']<=uqvalue)

ukb_shap['Risk_Group']=''
ukb_shap['Risk_Group'][mask_lq]='3. Low'
ukb_shap['Risk_Group'][mak_iqr]='2. Medium'
ukb_shap['Risk_Group'][mask_hq]='1. High'

ukb_shap['risk_score_index']=ukb_shap['risk_score_norm']/ukb_shap['risk_score_norm'].mean()
ukb_shap['risk_score_index_mod']=ukb_shap['risk_score_norm_mod']/ukb_shap['risk_score_norm_mod'].mean()

In [ ]:
mask_sleep=(ukb_shap_geno['sleep_duration_f1160'].between(5,10))
lines_conf('sleep_duration_f1160',df=ukb_shap_geno[mask_sleep],minsamp=200,rounded=10,regplot=0,e4e4=1,riskscore=0)
pvalue_slopes(ukb_shap_geno[mask_sleep],'sleep_duration_f1160')